In [ ]:
import numpy as np
from os import urandom
from  keras. callbacks  import  ModelCheckpoint, LearningRateScheduler
from  keras. models  import  Model
from  keras. layers  import  Dense, Conv1D, Conv2D,Input, ReLU,Reshape, Permute, Add, Flatten, BatchNormalization, Activation, Dropout,DepthwiseConv2D
from keras.regularizers import l2
import  matplotlib. pyplot  as  plt
import time
from  keras  import  layers
import tensorflow as tf
import copy

In [ ]:
Sbox = np.array([0xc, 0x5, 0x6, 0xb, 0x9, 0x0, 0xa, 0xd, 0x3, 0xe, 0xf, 0x8, 0x4, 0x7, 0x1, 0x2])

raw_P = [0,  16, 32, 48, 1,  17, 33, 49, 2,  18, 34, 50, 3,  19, 35, 51,
     4,  20, 36, 52, 5,  21, 37, 53, 6,  22, 38, 54, 7,  23, 39, 55,
     8,  24, 40, 56, 9,  25, 41, 57, 10, 26, 42, 58, 11, 27, 43, 59,
     12, 28, 44, 60, 13, 29, 45, 61, 14, 30, 46, 62, 15, 31, 47, 63]
raw_P = np.array(raw_P)

# Big-Edian
index = np.array([63 - i for i in range(64)])
raw_P = 63 - raw_P[index]

P = np.array([np.where(raw_P == i) for i in range(64)])
P = np.squeeze(P)

# for decryption, to be test
Sbox_inverse = np.array([0x5, 0xe, 0xf, 0x8, 0xc, 0x1, 0x2, 0xd, 0xb, 0x4, 0x6, 0x3, 0x0, 0x7, 0x9, 0xa])
P_inverse = raw_P

# for updating keys
KP = np.array([(i+61) % 80 for i in range(80)])


# x shape: (-1, 4)
def get_Sbox_output_enc(x):
    n, m = np.shape(x)
    assert m == 4
    x_val = x[:, 0] * 8 + x[:, 1] * 4 + x[:, 2] * 2 + x[:, 3]
    y_val = Sbox[x_val]
    output = np.zeros((n, 4), dtype=np.uint8)
    for i in range(4):
        output[:, i] = (y_val >> (3 - i)) & 1
    # print('y_val shape is ', np.shape(y_val))
    return output


# x shape: (-1, 4)
def get_Sbox_output_dec(x):
    n, m = np.shape(x)
    assert m == 4
    x_val = x[:, 0] * 8 + x[:, 1] * 4 + x[:, 2] * 2 + x[:, 3]
    y_val = Sbox_inverse[x_val]
    output = np.zeros((n, 4), dtype=np.uint8)
    for i in range(4):
        output[:, i] = (y_val >> (3 - i)) & 1
    # print('y_val shape is ', np.shape(y_val))
    return output


# keys shape: (-1, 80)
def update_master_key(keys, round_counter):
    tp = keys[:, KP]
    new_keys = copy.deepcopy(tp)
    new_keys[:, :4] = get_Sbox_output_enc(tp[:, :4])
    round_counter_arr = np.array([(round_counter >> (4-i)) & 1 for i in range(5) ], dtype=np.uint8)
    new_keys[:, 60:65] = tp[:, 60:65] ^ round_counter_arr
    return new_keys


# keys shape: (-1, 80)
def expand_key(keys, nr):
    n, m = np.shape(keys)
    assert m == 80
    ks = np.zeros((nr+1, n, 64), dtype=np.uint8)
    ks[0] = keys[:, :64]
    for i in range(1, nr+1):
        keys = update_master_key(keys, i)
        ks[i] = keys[:, :64]
    return ks


# x shape: (-1, 64)
def sBoxLayer_enc(x):
    n, m = np.shape(x)
    assert m == 64
    output = np.zeros((n, 64), dtype=np.uint8)
    for i in range(16):
        st = 4 * i
        output[:, st:st+4] = get_Sbox_output_enc(x[:, st:st+4])
    return output


# x shape: (-1, 64)
def sBoxLayer_dec(x):
    n, m = np.shape(x)
    assert m == 64
    output = np.zeros((n, 64), dtype=np.uint8)
    for i in range(16):
        st = 4 * i
        output[:, st:st+4] = get_Sbox_output_dec(x[:, st:st+4])
    return output


# x shape: (-1, 64)
def pLayer_enc(x):
    output = x[:, P]
    return output


# x shape: (-1, 64)
def pLayer_dec(x):
    output = x[:, P_inverse]
    return output


# x shape: (-1, 64)
# subkeys shape: (-1, 64)
def enc_one_round(x, subkeys):
    y = sBoxLayer_enc(x)
    z = pLayer_enc(y)
    output = z ^ subkeys
    return output
def dec_one_round(x, subkeys):
    y = pLayer_dec(x)
    z = sBoxLayer_dec(y)
    output = z ^ subkeys
    return output


def encrypt(x, ks):
    nr = ks.shape[0]
    y = x ^ ks[0]
    for i in range(1, nr):
        y = enc_one_round(y, ks[i])
    return y
def decrypt(x, ks):
    nr = ks.shape[0]
    y = x ^ ks[nr-1]
    for i in range(1, nr):
        y = dec_one_round(y, ks[nr - 1 - i])
    return y
def decrypt_1(x):
    y = pLayer_dec(x)
    z = sBoxLayer_dec(y)
    return y,z
def decrypt_2(x):
    y = pLayer_dec(x)
#     z = sBoxLayer_dec(y)
    return y


def make_train_data(subkeyy,n=10**7, nr=9):
    x0 = np.frombuffer(urandom(n * 8), dtype=np.uint64)  # .reshape(-1, 1)
    p0 = np.zeros((n, 64), dtype=np.uint8)
    for i in range(64):
        off = 63 - i
        p0[:, i] = (x0 >> off) & 1
    arr = [[0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 1], [0, 1, 0, 0], [0, 1, 0, 1], [0, 1, 1, 0],
           [0, 1, 1, 1], [1, 0, 0, 0], [1, 0, 0, 1], [1, 0, 1, 0], [1, 0, 1, 1], [1, 1, 0, 0], [1, 1, 0, 1],
           [1, 1, 1, 0], [1, 1, 1, 1]]
#     master_keys = np.frombuffer(urandom(n * 80), dtype=np.uint8).reshape(-1, 80) & 1
    subkeys = expand_key(subkeyy, nr)
    c0 = encrypt(p0, subkeys)

 

    #生成正样本
    pp_1=p0.copy()
    for j in range(4):
        pp_1[:,j]=pp_1[:,j]^arr[0][j]
    c0_1 = encrypt(pp_1, subkeys)
    c0_1_xor=np.bitwise_xor(c0,c0_1)

#     c0_0=decrypt_1(c0)
    c0_1_xor_2,c0_1_xor=decrypt_1(c0_1)
#     creal=c0_1_xor.copy()
#     creal_1=c0_1_xor_2.copy()
    creal=np.concatenate((c0,c0_1),axis=1) #解密一轮在异或
#     creal_1=np.concatenate((c0_1,c0_1_xor),axis=1)
    # creal_0=np.concatenate((c0,c0_1),axis=1) #解密一轮直接做
    for i in range(1,14,2):
        pp=p0.copy()
        pp_1=p0.copy()
        for j in range(4):
            pp[:,j]=pp[:,j]^arr[i][j]
            pp_1[:,j]=pp_1[:,j]^arr[i+1][j]
        c1=encrypt(pp, subkeys)
        c1_1=encrypt(pp_1, subkeys)

        c1_xor=np.bitwise_xor(c0,c1)
        c1_1_xor=np.bitwise_xor(c0,c1_1)
        c1_xor_tran_2,c1_xor_tran=decrypt_1(c1)
        c1_1_xor_tran_2,c1_1_xor_tran=decrypt_1(c1_1)
        
        creal=np.concatenate((creal,c1,c1_1),axis=1)
#         creal_1=np.concatenate((creal,c1_xor_tran_2,c1_1_xor_tran_2),axis=1)

#         creal=np.concatenate((creal,np.bitwise_xor(c0_0,c1_xor),np.bitwise_xor(c0_0,c1_1_xor)),axis=1)#解密一轮在异或
#         creal_1=np.concatenate((creal_1,np.bitwise_xor(c0_1,c1),np.bitwise_xor(c0_1,c1_1)),axis=1)
        # creal_0=np.concatenate((creal_0,c1,c1_1),axis=1)#解密一轮直接做
    #生成负样本
    x1 = np.frombuffer(urandom(n * 8), dtype=np.uint64)
    p1 = np.zeros((n, 64), dtype=np.uint8)
    x1_1 = np.frombuffer(urandom(n * 8), dtype=np.uint64)
    p1_1 = np.zeros((n, 64), dtype=np.uint8)
    for i in range(64):
        off = 63 - i
        p1[:, i] = (x1 >> off) & 1
        p1_1[:, i] = (x1_1 >> off) & 1
    c2=encrypt(p1, subkeys)
    c2_1=encrypt(p1_1, subkeys)

    c2_1_xor=np.bitwise_xor(c2,c2_1)

#     c2_2=decrypt_1(c2)
    c2_1_xor_2,c2_1_xor=decrypt_1(c2_1)
#     crand=c2_1_xor.copy()
#     crand_2=c2_1_xor_2.copy()
    crand=np.concatenate((c2,c2_1),axis=1)  #异或值
#     crand_1=np.concatenate((c2_1,np.bitwise_xor(c2,c2_1)),axis=1)  #异或值
    # crand_0=np.concatenate((c2,c2_1),axis=1)
    for j in range(7):
        x1 = np.frombuffer(urandom(n * 8), dtype=np.uint64)
        p1 = np.zeros((n, 64), dtype=np.uint8)
        x1_1 = np.frombuffer(urandom(n * 8), dtype=np.uint64)
        p1_1 = np.zeros((n, 64), dtype=np.uint8)
        for i in range(64):
            off = 63 - i
            p1[:, i] = (x1 >> off) & 1
            p1_1[:, i] = (x1_1 >> off) & 1
        c3=encrypt(p1, subkeys)
        c3_1=encrypt(p1_1, subkeys)

        c3_xor=np.bitwise_xor(c2,c3)
        c3_1_xor=np.bitwise_xor(c2,c3_1)

        c3_xor_tran_2,c3_xor_tran=decrypt_1(c3)
        c3_1_xor_tran_2,c3_1_xor_tran=decrypt_1(c3_1)

        crand=np.concatenate((crand,c3,c3_1),axis=1)
#         crand_1=np.concatenate((crand_1,c3_xor_tran_2,c3_1_xor_tran_2),axis=1)
#         crand=np.concatenate((crand,np.bitwise_xor(c2_2,c3_xor),np.bitwise_xor(c2_2,c3_1_xor)),axis=1)
#         crand_1=np.concatenate((crand_1,np.bitwise_xor(c2_1,c3),np.bitwise_xor(c2_1,c3_1)),axis=1)
        # crand_0=np.concatenate((crand_0,c3,c3_1),axis=1)
    #生成负样本2


#     X = np.concatenate((creal_0,crand_0))
    X = np.concatenate((creal,crand))
    Yreal  =  np. ones(n)
    Yrand  =  np. zeros(n)
    Y  =  np. concatenate((Yreal, Yrand))
    return X,Y
# verify(n=10, nr=31)
def make_train_data_mutil(nums,rounds):
    master_keys = np.frombuffer(urandom(nums * 80), dtype=np.uint8).reshape(-1, 80) & 1
    X,Y=make_train_data(master_keys,nums,rounds)
#     X_1,Y_1=make_train_data(master_keys,nums,rounds)
#     X_2,Y_2,crd_2=make_train_data(master_keys,nums,rounds)
#     X_3,Y_3,crd_3=make_train_data(master_keys,nums,rounds)
#     X_test=np.concatenate((X,X_1),axis=1)
    return X,Y

In [ ]:
def MBConvBlock(inputs, expansion_ratio, stride, filters, kernel_size):
    # 定义扩展通道数
    expand_channels = expansion_ratio * inputs.shape[-1]
    
    # 第一层扩展卷积
    x = Conv2D(expand_channels, kernel_size=(1, 1), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    # 深度可分离卷积
    x = DepthwiseConv2D(kernel_size, strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x=Dropout(0.5)(x)
    # 线性投影回归
    x = Conv2D(filters, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
#     x = tf.keras.layers.Add()([x, inputs])
    
    return x

In [ ]:
bs = 2000
# 定义初始学习率、目标学习率和衰减步数
initial_lr = 0.002
final_lr = 0.0001
decay_epochs = 10

def decay_learning_rate(epoch, initial_lr, final_lr, decay_epochs):
    decay_factor = (final_lr / initial_lr) ** (1 / decay_epochs)
    current_lr = initial_lr * (decay_factor ** epoch)
    return current_lr




def cyclic_lr(epoch, high_lr):
    if epoch<=10:
        return 0.001 +  (0.0001*(10-epoch))
    elif 10<epoch<=25:
        return 0.001
    else:
        res=0.001 -  (0.0001*(epoch-25))
        if res>0.0001:
            return res
        return 0.0001
def make_checkpoint(file):
    res=ModelCheckpoint(file, monitor='val_loss', save_best_only=True)
    return (res)
#make residual tower of convolutional blocks
def make_resnet(num_words=16,multiset=16, num_filters=16, num_outputs=1, d1=1024, d2=1024, word_size=4, ks=3,depth=5, reg_param=0.00001, final_activation='sigmoid'):
  #Input and preprocessing layers
  inp = Input(shape=(1*num_words * word_size *multiset ,))
  rs = Reshape((4, 16,16))(inp)
  perm = Permute((2,3,1))(rs)
  input = Conv2D(num_filters , kernel_size=1, padding='same', kernel_regularizer=l2(reg_param))(perm)
  input = BatchNormalization()(input)
  input = Activation('relu')(input)
#densenet稠密神经网络
  for i in range(depth):
        conv1 = BatchNormalization()(input)
        conv1 = Activation('relu')(conv1)
        conv1 = MBConvBlock(conv1, expansion_ratio=2, stride=1, filters=num_filters, kernel_size=ks)
        conv2 = Conv2D(num_filters, kernel_size=ks, padding='same',kernel_regularizer=l2(reg_param))(conv1)
        input = tf.keras.layers.Concatenate()([conv2, input]) 
#         input = tf.keras.layers.Add()([conv2, input]) 
#         num_filters += 16

  conv3 = Conv2D(16, kernel_size=ks, padding='same',kernel_regularizer=l2(reg_param))(input)
  conv3 = BatchNormalization()(conv3)
#   conv3=Dropout(0.5)(conv3)
  conv3 = Activation('relu')(conv3)
  
  conv4 = Conv2D(8, kernel_size=ks, padding='same',kernel_regularizer=l2(reg_param))(conv3)
  conv4 = BatchNormalization()(conv4)
#   conv4=Dropout(0.5)(conv4)
  conv4 = Activation('relu')(conv4)
  
  conv5 = Conv2D(4, kernel_size=ks, padding='same',kernel_regularizer=l2(reg_param))(conv4)
  conv5 = BatchNormalization()(conv5)
#   conv5=Dropout(0.5)(conv5)
  conv5 = Activation('relu')(conv5)
    
  flat1 = Flatten()(conv5)
  dense1 = Dense(1024,kernel_regularizer=l2(reg_param))(flat1)
  dense1 = BatchNormalization()(dense1)
  dense1 = Activation('relu')(dense1)
  dense2 = Dense(1024, kernel_regularizer=l2(reg_param))(dense1)
  dense2 = BatchNormalization()(dense2)
  dense2 = Activation('relu')(dense2)
#   dense3=Dense(d1,kernel_regularizer=l2(reg_param))(dense1)
#   dense3 = BatchNormalization()(dense3)
#   dense3 = Activation('relu')(dense3)
#   dense4=Dense(d1,kernel_regularizer=l2(reg_param))(dense3)
#   dense4 = BatchNormalization()(dense4)
#   dense4 = Activation('relu')(dense4)
  dense2=Dropout(0.5)(dense2)
  out = Dense(num_outputs, activation=final_activation, kernel_regularizer=l2(reg_param))(dense2)
  model = Model(inputs=inp, outputs=out)
  return(model)
def train_present_distinguisher(num_epochs, num_rounds, depth):
    # create the network
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    except ValueError:
        tpu = None

    # TPUStrategy for distributed training
    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("train on tpu")
    else: # default strategy that works on CPU and single GPU
      strategy = tf.distribute.get_strategy()
    with strategy.scope():
        net = make_resnet(depth=depth, reg_param=0.00005)
        net.compile(optimizer='adam', loss='mse', metrics=['acc'])
    # generate training and validation data and test data
    print("训练数据生成中。。。")
    X,Y=make_train_data_mutil(2**20,num_rounds)
    print("训练数据生成完成，生成验证数据中。。。")
    X_eval,Y_eval=make_train_data_mutil(2**16,num_rounds)
#     X=np.load("/kaggle/input/pressent-def-p/train_data_1.npy")
#     Y = np.load("/kaggle/input/pressent-def-p/train_data_lbel_1.npy")
#     X_eval=np.load("/kaggle/input/pressent-def-p/test_data.npy"),
#     Y_eval = np.load("/kaggle/input/pressent-def-p/test_data_lbel.npy")
    print("验证数据生成完成，全部数据生成完成，开始训练")
    # X_test, Y_test = make_train_data(2 ** 8, num_rounds)
    # create learnrate schedule
#     lr_scheduler = lambda epoch: decay_learning_rate(epoch, initial_lr, final_lr, decay_epochs)
    lr = LearningRateScheduler(cyclic_lr)
    # train and evaluate

    
    time_start = time.time()
    
    h  =  net.fit(X, Y, epochs=num_epochs, batch_size=bs, validation_data=(X_eval, Y_eval), callbacks=[lr])
    
    time_end = time.time()
    total_time = time_end - time_start
    # loss, accuracy  =  net. evaluate(X_test, Y_test)
    print("\nWhen training for a", num_rounds, "round PRESENT ", num_epochs, "epochs:")
    print("\nBest validation accuracy: ", np.max(h.history['val_acc']))
    # print('\nTest loss:', loss)
    # print('\nTest accuracy:', accuracy)
    # f = open(save_path + "result_for_lyu_train_PRESENT.txt", "a")
    print('\nTotal training time is: %.2f seconds.' % total_time)
    return (net, h)

In [ ]:
model,history=train_present_distinguisher(50, 5, 10)
acc  =  history. history['acc']
val_acc  =  history. history['val_acc']
epochs = range(1, len(acc) + 1)
plt. figure(figsize=(6, 4))
plt. plot(epochs, acc, 'b', label='Training accuracy')
plt. plot(epochs, val_acc, 'g', label='Validation accuracy')
plt. title('Training and validation accuracy')
plt. xlabel('Epochs')
plt. ylabel('Accuracy')
plt. legend(loc='upper left')
plt. show()

In [ ]:
X_test, Y_test = make_train_data_mutil(2 ** 16, 5)
loss, accuracy  =  model. evaluate(X_test, Y_test)
print('\nTest loss:', loss)
print('\nTest accuracy:', accuracy)